# 🦜🔗 LangChain agent interacting with gmail.

Langchain stellt eine ganze Palette an Toolkits bereit die es Agenten ermöglicht, mit externen Services wie zum Beispiel Gmail, Office365, etc. zu interagieren.
Ein Toolkit ist im Grunde eine Sammlung mehrerer Tools, die man gesammelt einem Agenten zur Verfügung stellt werden können.

In diesem Beispiel wollen wir uns das GmailToolkit ansehen.


Zuerst müssen wird das Toolkit konfigurieren, sodass es per Gmail API mit unserem Google Konto kommunizieren kann.


In [7]:
from langchain_community.tools.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="credentials.json",
)
api_resource = build_resource_service(credentials=credentials)

Jetzt initialisieren wird das Toolkit und übergeben die vorher konfigurierten API Settings.

Im Output sehen wir, welche Tools über das Toolkit bereit gestellt werden.


In [8]:
from langchain_community.agent_toolkits import GmailToolkit
from langgraph.prebuilt import ToolExecutor

toolkit = GmailToolkit(api_resource=api_resource)
tools = toolkit.get_tools()
tools_executor = ToolExecutor(tools)
tools

/var/folders/01/ywnzdvn56c16b5vdy9c7_q8w0000gn/T/ipykernel_22666/462995026.py:6: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tools_executor = ToolExecutor(tools)


[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x1694ea310>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x1694ea310>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x1694ea310>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x1694ea310>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x1694ea310>)]

Letzlich erstellen wir einen Langchain Graphen. Hier nutzen wir wieder einen prebuilt Graphen, den wir in diesem Schritt auch ausführen.


In [9]:
from langchain.schema import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent

from helpers import llm

prompt = SystemMessage(content="You are an email writer assistant.")
app = create_react_agent(llm(), tools, state_modifier=prompt)

inputs = {
    "messages": [
        HumanMessage(
            content="""Create an email draft to donald@trumppp.comm about a contract building a pyramid in Egypt.
            Please argue why we are the best company for the job.
            Under no circumstances may you send the message, however. """
        )
    ]
}


app.invoke(inputs)

{'messages': [HumanMessage(content='Create an email draft to donald@trumppp.comm about a contract building a pyramid in Egypt.\n            Please argue why we are the best company for the job.\n            Under no circumstances may you send the message, however. ', id='b95bc96e-d121-46ab-ab5d-7465a00e9370'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_WyO3i9Oxt96eJljjNwoEmOzd', 'function': {'arguments': '{"message":"Subject: Proposal for Pyramid Construction in Egypt\\n\\nDear Donald,\\n\\nI hope this message finds you well. I am writing to discuss an exciting opportunity for a contract to build a pyramid in Egypt, and I would like to present why our company is the best fit for this monumental task.\\n\\nOur company has a long-standing reputation for excellence in large-scale construction projects, particularly in culturally significant structures. With over two decades of experience, we have successfully completed numerous projects that blend t